In [ ]:
import datetime

from definitions import ROOT_DIR
from tinyman.v1.client import TinymanMainnetClient
from algo.blockchain.process_volumes import query_transactions, SwapScraper
from algo.blockchain.process_prices import PriceScraper
from algo.blockchain.utils import datetime_to_int, generator_to_df
import json
import itertools
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from definitions import ROOT_DIR
import numpy as np
import glob
import os
from matplotlib import pyplot as plt
from pathlib import Path
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from algo.universe.assets import get_asset_data
from algo.blockchain.utils import timestamp_to_5min
from functools import lru_cache
from definitions import ROOT_DIR

In [ ]:
cache_name = '20220206'

# Select only pairs with Algo
caches_pattern = f'{ROOT_DIR}/caches/prices/{cache_name}/0_*'

stablecoins = [31566704, # USDC
               312769 # USDt
              ]

@lru_cache()
def get_asset_name(asset_id):
    return get_asset_data(asset_id)['params']['name']

@lru_cache()
def get_decimals(asset_id):
    return get_asset_data(asset_id)['params']['decimals']

def gen_data():
    for base_dir in glob.glob(caches_pattern):
        asset_id = int(Path(base_dir).name.split('_')[1])
        if asset_id in stablecoins:
            continue
        subdf = pd.concat(pd.read_parquet(fname) for fname in glob.glob(f'{base_dir}/*.parquet')).sort_values(by='time')
        subdf['id'] = asset_id
        yield subdf
    
df = pd.concat(gen_data())

df['time_5min'] = timestamp_to_5min(df['time'])
df = df.groupby(['time_5min','id']).agg(sum)
df['price'] = df['asset2_reserves']/df['asset1_reserves']
df = df.reset_index()

In [ ]:
keyname = 'id'
keys = list(df[keyname].unique())

f,axs = plt.subplots(len(keys),1,figsize=(10,40), sharex='col')

for (asset_id,group),ax in zip(df.groupby(['id']), axs):
    asset_name = get_asset_name(asset_id)
    decimals = get_decimals(asset_id)

    ax.plot(group['time_5min'], group['price'] * 10**(decimals-6), label=asset_id);
    ax.set_title(asset_name);
    ax.legend();
    ax.grid();
    
plt.xticks(rotation=35);